## Global analysis of 670nm excitation TA of WL-PSI of SCy6803

### Inspect data

In [ ]:
from cycler import cycler
from glotaran.io import load_parameters, save_result
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme
from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.style import PlotStyle
from pyglotaran_extras.plotting.plot_traces import (
    plot_fitted_traces,
    select_plot_wavelengths,
)
from pyglotaran_extras.inspect import show_a_matrixes

In [ ]:
from pyglotaran_extras import plot_data_overview

DATA_PATH1 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_reva.ascii"
DATA_PATH2 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revb.ascii"
DATA_PATH3 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revc.ascii"
DATA_PATH4 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revd.ascii"
fig, axes = plot_data_overview(
    DATA_PATH1, nr_of_data_svd_vectors=5, linlog=False,  cmap='seismic',vmin = -7,vmax=7,use_svd_number=True
)
# change color map seismic or bwr
# axes[0].set_cmap('seismic')

In [ ]:
import matplotlib.pyplot as plt
# import matplotlib.pyplot.set_cmap as set_cmap
plt.set_cmap('seismic')
fig, axes = plot_data_overview(
    DATA_PATH2, nr_of_data_svd_vectors=5, linlog=False, linthresh=10,  cmap='seismic',vmin = -7,vmax=7,use_svd_number=True
)


## Global Analysis

### Used model and parameters

In [ ]:
global_model_path = "models/20230821model_PSI_TA_SCy6803WLglobal670_with_DOAS.yml"

In [ ]:
# global_parameters_path = "models/20230821optimized_parameters670_with_DOAS.csv"
# optimizedparameters = load_parameters(global_parameters_path)

In [ ]:
# global_parameters_path = "models/20230821optimized_parameters670_with_DOAS.csv"
optimizedparameters = load_parameters("models/20230821optimized_parameters670_with_DOAS.csv")

#### Model file

In [ ]:
# Uncomment the following 2 lines to display the global model file in the notebook
# from glotaran.utils.ipython import display_file
# display_file(global_model_path, syntax="yaml")

# Alternatively (recommended), open the file in a text editor to see the model definition

#### Parameters file

In [ ]:
# Uncomment the next line and run the cell to print the starting values of the analysis
# These starting values have already been optimized, hence the name optimizedparameters

# optimizedparameters

### Create scheme and optimize it

In [ ]:
global_scheme = Scheme(
    model="models/20230821model_PSI_TA_SCy6803WLglobal670_without_DOAS.yml",  # type: ignore
    parameters=load_parameters("models/20230821optimized_parameters670_without_DOAS.csv"),
    maximum_number_function_evaluations=15,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "670TR1": DATA_PATH1,
        "670TR2": DATA_PATH2,
    },  # type: ignore
)
global_scheme.validate()

In [ ]:
global_result1 = optimize(global_scheme, raise_exception=True)

## Residual analysis of the 670 nm excitation TR1 data

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_residual import plot_residual
from pyglotaran_extras.plotting.plot_svd import plot_lsv_residual
from pyglotaran_extras.plotting.plot_svd import plot_rsv_residual


def plot_residual_and_svd(result_dataset):
    fig, axes = plt.subplots(1, 3, figsize=(10, 2))
    plot_residual(result_dataset, axes[0])
    axes[0].get_legend().remove()
    axes[0].set_ylabel("Wavelength (nm)")
    plot_lsv_residual(result_dataset, axes[1], indices=[0])
    axes[1].get_legend().remove()
    axes[1].set_ylabel("")
    axes[1].set_title("residual 1st LSV")
    plot_rsv_residual(result_dataset, axes[2], indices=[0])
    axes[2].set_xlabel("Wavelength (nm)")
    axes[2].set_title("residual 1st RSV")
    axes[2].get_legend().remove()
    axes[2].set_ylabel("")

    return fig, axes


fig, axes = plot_residual_and_svd(global_result1.data["670TR1"])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

# Plot result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_das
from pyglotaran_extras.plotting.plot_spectra import plot_sas
from pyglotaran_extras.plotting.style import ColorCode

myFRLcolors = [ "tab:grey","tab:orange",  ColorCode.cyan, ColorCode.green,"m", "y", "k","r", "b", "tab:purple"]

custom_cycler = cycler(color=myFRLcolors)


def plot_concentration_and_spectra(result_dataset,result_dataset2):
    # fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    plot_concentrations(result_dataset, axes[0], center_λ=0, linlog=True, cycler=custom_cycler)
    plot_concentrations(result_dataset2, axes[0], center_λ=0, linlog=True, cycler=custom_cycler)
    plot_sas(result_dataset, axes[1], cycler=custom_cycler)
    plot_das(result_dataset, axes[2], cycler=custom_cycler)
    return fig, axes


fig, axes = plot_concentration_and_spectra(global_result1.data["670TR1"],global_result1.data["670TR2"])
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("EADS (mOD)")
axes[1].set_title("EADS")
axes[2].set_xlabel("Wavelength (nm)")
axes[2].set_ylabel("DADS (mOD)")
axes[2].set_title("DADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

In [ ]:
global_scheme = Scheme(
    model="models/20230821model_PSI_TA_SCy6803WLglobal670_with_DOAS.yml",  # type: ignore
    parameters=load_parameters("models/20230821optimized_parameters670_with_DOAS.csv"),
    maximum_number_function_evaluations=15,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "670TR1": DATA_PATH1,
        "670TR2": DATA_PATH2,
    },  # type: ignore
)
global_scheme.validate()

In [ ]:
global_result = optimize(global_scheme, raise_exception=True)

In [ ]:
global_result670=global_result

## Residual analysis of the 670 nm excitation TR1 data

In [ ]:
# import matplotlib.pyplot as plt
# from pyglotaran_extras.plotting.plot_residual import plot_residual
# from pyglotaran_extras.plotting.plot_svd import plot_lsv_residual
# from pyglotaran_extras.plotting.plot_svd import plot_rsv_residual


# def plot_residual_and_svd(result_dataset):
#     fig, axes = plt.subplots(1, 3, figsize=(10, 2))
#     plot_residual(result_dataset, axes[0])
#     axes[0].get_legend().remove()
#     axes[0].set_ylabel("Wavelength (nm)")
#     plot_lsv_residual(result_dataset, axes[1], indices=[0])
#     axes[1].get_legend().remove()
#     axes[1].set_ylabel("")
#     axes[1].set_title("residual 1st LSV")
#     plot_rsv_residual(result_dataset, axes[2], indices=[0])
#     axes[2].set_xlabel("Wavelength (nm)")
#     axes[2].set_title("residual 1st RSV")
#     axes[2].get_legend().remove()
#     axes[2].set_ylabel("")

#     return fig, axes


fig, axes = plot_residual_and_svd(global_result.data["670TR1"])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

# Plot result for interpretation


In [ ]:
# import matplotlib.pyplot as plt
# from cycler import cycler
# from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
# from pyglotaran_extras.plotting.plot_spectra import plot_das
# from pyglotaran_extras.plotting.plot_spectra import plot_sas
# from pyglotaran_extras.plotting.style import ColorCode

# myFRLcolors = [ "tab:grey","tab:orange",  ColorCode.cyan, ColorCode.green,"m", "y", "k","r", "b", "tab:purple"]

# custom_cycler = cycler(color=myFRLcolors)


# def plot_concentration_and_spectra(result_dataset,result_dataset2):
#     # fig, axes = plt.subplots(1, 2, figsize=(18, 7))
#     fig, axes = plt.subplots(1, 3, figsize=(15, 4))
#     plot_concentrations(result_dataset, axes[0], center_λ=0, linlog=True, cycler=custom_cycler)
#     plot_concentrations(result_dataset2, axes[0], center_λ=0, linlog=True, cycler=custom_cycler)
#     plot_sas(result_dataset, axes[1], cycler=custom_cycler)
#     plot_das(result_dataset, axes[2], cycler=custom_cycler)
#     return fig, axes


fig, axes = plot_concentration_and_spectra(global_result.data["670TR1"],global_result.data["670TR2"])
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("EADS (mOD)")
axes[1].set_title("EADS")
axes[2].set_xlabel("Wavelength (nm)")
axes[2].set_ylabel("DADS (mOD)")
axes[2].set_title("DADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

To save the results of the optimization we can use the `save_result` command.

Because it saves *everything* it consumes about 50MB of disk space per save.

In [ ]:
save_result(
    result=global_result,
    result_path="results/20230830/result.yaml",
    allow_overwrite=True,
)

15         5.7843e+02 linking k54, estimated 28.021 ps, rmse 6.74e-02
15      5.7931e+02 linking k54 at the 700 nm estimate of 1/26 ps, rmse 6.75e-02 timu 6.666e-2 because of 670TR1 being smaller 6.975 instead of 7.29
15         5.7048e+02  free DOAS rate
15         5.6983e+02 free DOAS rate&freq rmse 6.69e-02 670TR1 7.04e-02 but IRFAS & DOAS compensate
-   15         6.1658e+02  no DOAS
-   15         5.6870e+02  with DOAS & CAwidth IRFAS & DOAS peak at 670
-   6         5.6838e+02 26.45 ps

### Results and parameters

In [ ]:
# Just call the result to get the optimization result summary.
global_result
# For easier copy-and-paste try:
# print(global_result)

In [ ]:
# Access the result's `optimized_parameters` to print a markdown table of the optimized parameters:
global_result.optimized_parameters

# what to do if the estimated rate constants are not in decreasing order?
then one should sort the estimated rate constants in decreasing order, and repeat the fit with those new starting values

### Amplitude matrices

In [ ]:
show_a_matrixes(global_result)

# Result plots

<sub>Note: The color scheme of the plots in this notebook may not match published figures.</sub>

## Fit quality

In [ ]:
global_result_TA = (
    global_result.data["670TR1"],
    global_result.data["670TR2"],
)
wavelengths = select_plot_wavelengths(global_result_TA, equidistant_wavelengths=True)
plot_fitted_traces(global_result_TA, wavelengths, linlog=True, linthresh=1);

The above command `plot_fitted_traces` is used to plot a selection of traces for a set of wavelengths (autogenerated using the `select_plot_wavelengths` function).
To show to make a manual selection of traces, and 'dress up the plot' see the code below, which reproduces Figure 2 of the paper.

In [ ]:
# Reproduction of Figure 2 of the paper
import warnings
from pyglotaran_extras.plotting.style import ColorCode as cc

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fig, ax_ = plot_fitted_traces(
        global_result_TA,
        [685, 700, 720, 760],
        linlog=True,
        linthresh=1,  # published figure uses 0.3 for easthetic reasons, but here 1 looks better
        axes_shape=(2, 2),
        figsize=(6, 4),
        title="",
        per_axis_legend=True,
        cycler=cycler(
            color=[
                cc.grey,
                cc.black,
                cc.grey,
                cc.black,
                cc.orange,
                cc.red,
                cc.orange,
                cc.red,
            ]
        ),
    )
    handles, labels = ax_.flatten()[0].get_legend_handles_labels()
    for i in range(len(handles)):
        if i == 1:
            labels[i] = "670 nm excitation"
        elif i == 5:
            labels[i] = "700 nm excitation"
        else:
            labels[i] = "_Hidden"
    for idx, ax in enumerate(ax_.flatten()):
        ax.set_ylabel(ax.title.get_text().replace("spectral = ", ""))
        if idx > 1:
            ax.set_xlabel("Time (ps)")
        else:
            ax.set_xlabel("")
        ax.set_title("")
        if ax.get_legend() is not None:
            ax.get_legend().remove()
        for line in ax.lines:
            line.set_linewidth(0.5)  # Set the line width here
    fig.legend(
        handles,
        labels,
        bbox_to_anchor=(0.5, -0.05),
        loc="lower center",
        ncol=len(handles),
    )
    fig.tight_layout()

## Overview 670 exc

In [ ]:
plot_overview(
    global_result.data["670TR1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=["y", "g", "tab:orange", "r", "k", "c", "b", "m", "tab:purple"]
    ),use_svd_number=True,svd_cycler=PlotStyle().cycler,
);

In [ ]:
plot_overview(
    global_result.data["670TR2"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=["y", "g", "tab:orange", "r", "k", "c", "b", "m", "tab:purple"]
    ),use_svd_number=True,svd_cycler=PlotStyle().cycler,
);

## Coherent Artifact


In [ ]:
from pyglotaran_extras import plot_coherent_artifact

fig, axes = plot_coherent_artifact(global_result.data["670TR1"], time_range=(-0.3, 0.3), figsize=(10, 4))
axes[0].set_xlabel("Time (ps)")
axes[1].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("")
axes[0].annotate("A", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
fig.tight_layout()

## Overview of the estimated DOAS and phases of 670 nm excitation data

In [ ]:
from pyglotaran_extras import plot_doas
from pyglotaran_extras.plotting.style import ColorCode

fig, axes = plot_doas(
    global_result.data["670TR1"],
    damped_oscillation=["osc1"],
    time_range=(-0.3, 0.3),
    spectral=700,
    figsize=(15, 4),
    normalize=False
    # oscillation_type="sin",
    # center_λ=550,
)

# for vline_pos in [415, 460]:
#     axes[1].axvline(vline_pos, color="r", linewidth=1)
#     axes[2].axvline(vline_pos, color="r", linewidth=1)
# for vline_pos in [526]:
#     axes[1].axvline(vline_pos, color=ColorCode.green, linewidth=1)
#     axes[2].axvline(vline_pos, color=ColorCode.green, linewidth=1)
# for vline_pos in [393, 429, 479]:
#     axes[1].axvline(vline_pos, color="b", linewidth=1)
#     axes[2].axvline(vline_pos, color="b", linewidth=1)
axes[0].set_xlabel("Time (ps)")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[2].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[1].set_title("DOAS")
# axes[0].annotate("A", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
# axes[1].annotate("B", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
# axes[2].annotate("C", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[0].annotate("C", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[1].annotate("D", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[2].annotate("E", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)

## Global analysis of 700nm excitation TA of WL-PSI of SCy6803

### Inspect data

In [ ]:
from cycler import cycler
from glotaran.io import load_parameters, save_result
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme
from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.plot_traces import (
    plot_fitted_traces,
    select_plot_wavelengths,
)
from pyglotaran_extras.inspect import show_a_matrixes

In [ ]:
from pyglotaran_extras import plot_data_overview

DATA_PATH3 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revc.ascii"
DATA_PATH4 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revd.ascii"
fig, axes = plot_data_overview(
    DATA_PATH3, nr_of_data_svd_vectors=5, linlog=False,  cmap='seismic',vmin = -7,vmax=7,use_svd_number=True
)
# change color map seismic or bwr
# axes[0].set_cmap('seismic')

In [ ]:
fig, axes = plot_data_overview(
    DATA_PATH4, nr_of_data_svd_vectors=5, linlog=False, linthresh=10,  cmap='seismic',vmin = -7,vmax=7,use_svd_number=True
)

## Global Analysis

### Used model and parameters

In [ ]:
global_model_path = "models/20230821model_PSI_TA_SCy6803WLglobal700.yml"

In [ ]:
global_parameters_path = "models/20230821optimized_parameters700.csv"
optimizedparameters = load_parameters(global_parameters_path)

#### Model file

In [ ]:
# Uncomment the following 2 lines to display the global model file in the notebook
# from glotaran.utils.ipython import display_file
# display_file(global_model_path, syntax="yaml")

# Alternatively (recommended), open the file in a text editor to see the model definition

#### Parameters file

In [ ]:
# Uncomment the next line and run the cell to print the starting values of the analysis
# These starting values have already been optimized, hence the name optimizedparameters

# optimizedparameters

### Create scheme and optimize it

In [ ]:
global_scheme = Scheme(
    model=global_model_path,  # type: ignore
    parameters=optimizedparameters,
    maximum_number_function_evaluations=15,
    clp_link_tolerance=0.1,
    data={
        # TA data
        # "670TR1": DATA_PATH1,
        # "670TR2": DATA_PATH2,
        "700TR1": DATA_PATH3,
        "700TR2": DATA_PATH4,
    },  # type: ignore
)
global_scheme.validate()

In [ ]:
global_result = optimize(global_scheme, raise_exception=True)

To save the results of the optimization we can use the `save_result` command.

Because it saves *everything* it consumes about 50MB of disk space per save.

In [ ]:
global_result700=global_result

In [ ]:
save_result(
    result=global_result,
    result_path="results/20230821/result.yaml",
    allow_overwrite=True,
)

-   15         2.6310e+02   no DOAS first k 120
-   12         2.7452e+02 four k
-   15         5.6870e+02  with DOAS & CAwidth IRFAS & DOAS peak at 670
-   6         5.6838e+02 26.45 ps

### Results and parameters

In [ ]:
# Just call the result to get the optimization result summary.
global_result
# For easier copy-and-paste try:
# print(global_result)

In [ ]:
# Access the result's `optimized_parameters` to print a markdown table of the optimized parameters:
global_result.optimized_parameters

# what to do if the estimated rate constants are not in decreasing order?
then one should sort the estimated rate constants in decreasing order, and repeat the fit with those new starting values

### Amplitude matrices

In [ ]:
show_a_matrixes(global_result)

### Amplitude matrices

In [ ]:
show_a_matrixes(global_result)

# Result plots

<sub>Note: The color scheme of the plots in this notebook may not match published figures.</sub>

## Fit quality

In [ ]:
global_result_TA = (
    global_result.data["700TR1"],
    global_result.data["700TR2"],
)
wavelengths = select_plot_wavelengths(global_result_TA, equidistant_wavelengths=True)
plot_fitted_traces(global_result_TA, wavelengths, linlog=True, linthresh=1);

The above command `plot_fitted_traces` is used to plot a selection of traces for a set of wavelengths (autogenerated using the `select_plot_wavelengths` function).
To show to make a manual selection of traces, and 'dress up the plot' see the code below, which reproduces Figure 2 of the paper.

In [ ]:
# Reproduction of Figure 2 of the paper
import warnings
from pyglotaran_extras.plotting.style import ColorCode as cc

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fig, ax_ = plot_fitted_traces(
        global_result_TA,
        [685, 700, 720, 760],
        linlog=True,
        linthresh=1,  # published figure uses 0.3 for easthetic reasons, but here 1 looks better
        axes_shape=(2, 2),
        figsize=(6, 4),
        title="",
        per_axis_legend=True,
        cycler=cycler(
            color=[
                cc.grey,
                cc.black,
                cc.grey,
                cc.black,
                cc.orange,
                cc.red,
                cc.orange,
                cc.red,
            ]
        ),
    )
    handles, labels = ax_.flatten()[0].get_legend_handles_labels()
    for i in range(len(handles)):
        if i == 1:
        #     labels[i] = "670 nm excitation"
        # elif i == 5:
            labels[i] = "700 nm excitation"
        else:
            labels[i] = "_Hidden"
    for idx, ax in enumerate(ax_.flatten()):
        ax.set_ylabel(ax.title.get_text().replace("spectral = ", ""))
        if idx > 1:
            ax.set_xlabel("Time (ps)")
        else:
            ax.set_xlabel("")
        ax.set_title("")
        if ax.get_legend() is not None:
            ax.get_legend().remove()
        for line in ax.lines:
            line.set_linewidth(0.5)  # Set the line width here
    fig.legend(
        handles,
        labels,
        bbox_to_anchor=(0.5, -0.05),
        loc="lower center",
        ncol=len(handles),
    )
    fig.tight_layout()

## Overview 700 exc

In [ ]:
myFRLcolors = ["k","r", "g", "tab:purple"]
custom_cycler = cycler(color=myFRLcolors)
plot_overview(
    global_result.data["700TR1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=1,
    cycler=custom_cycler,use_svd_number=True,svd_cycler=PlotStyle().cycler
);

In [ ]:
plot_overview(
    global_result.data["700TR2"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=10,
    cycler=custom_cycler,use_svd_number=True,svd_cycler=PlotStyle().cycler
);

## Coherent Artifact


In [ ]:
from pyglotaran_extras import plot_coherent_artifact

fig, axes = plot_coherent_artifact(global_result.data["700TR1"], time_range=(-0.3, 0.3), figsize=(10, 4))
axes[0].set_xlabel("Time (ps)")
axes[1].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("")
axes[0].annotate("A", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
fig.tight_layout()

# Plot result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_das
from pyglotaran_extras.plotting.plot_spectra import plot_sas
from pyglotaran_extras.plotting.style import ColorCode

# myFRLcolors = [ "tab:grey","tab:orange",  ColorCode.cyan, ColorCode.green,"m", "y", "k","r", "b", "tab:purple"]
myFRLcolors = ["k","r", "g", "tab:purple"]

custom_cycler = cycler(color=myFRLcolors)


def plot_concentration_and_spectra(result_dataset,result_dataset2):
    # fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    plot_concentrations(result_dataset, axes[0], center_λ=0, linlog=True, cycler=custom_cycler)
    plot_concentrations(result_dataset2, axes[0], center_λ=0, linlog=True, cycler=custom_cycler)
    plot_sas(result_dataset, axes[1], cycler=custom_cycler)
    plot_das(result_dataset, axes[2], cycler=custom_cycler)
    return fig, axes


fig, axes = plot_concentration_and_spectra(global_result.data["700TR1"],global_result.data["700TR2"])
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("EADS (mOD)")
axes[1].set_title("EADS")
axes[2].set_xlabel("Wavelength (nm)")
axes[2].set_ylabel("DADS (mOD)")
axes[2].set_title("DADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

In [ ]:
global_result700.data["700TR1"].species_associated_spectra[:,3];

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 4))
# plot_sas(global_result.data["700TR1"], axes[1], cycler=custom_cycler)
final670=global_result670.data["670TR1"].species_associated_spectra[:,4]
final700=0.85*global_result700.data["700TR1"].species_associated_spectra[:,3]
diff=final670-final700
# global_result700.data["700TR1"].species_associated_spectra[:,3].plot.line(x="spectral", ax=axes[0],color="purple")
final700.plot.line(x="spectral", ax=axes[0],color="k",label="700 exc")
# global_result670.data["670TR1"].species_associated_spectra[:,4].plot.line(x="spectral", ax=axes[0],color="magenta")
final670.plot.line(x="spectral", ax=axes[0],color="r",label="670 exc")
diff.plot.line(x="spectral", ax=axes[1])
axes[0].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("EADS (mOD)")
axes[0].set_title("final EADS")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("difference (mOD)")
axes[1].set_title("difference final EADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].legend()
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)



## Residual analysis of the 700 nm excitation TR1 data

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_residual import plot_residual
from pyglotaran_extras.plotting.plot_svd import plot_lsv_residual
from pyglotaran_extras.plotting.plot_svd import plot_rsv_residual


def plot_residual_and_svd(result_dataset):
    fig, axes = plt.subplots(1, 3, figsize=(10, 2))
    plot_residual(result_dataset, axes[0])
    axes[0].get_legend().remove()
    axes[0].set_ylabel("Wavelength (nm)")
    plot_lsv_residual(result_dataset, axes[1], indices=[0])
    axes[1].get_legend().remove()
    axes[1].set_ylabel("")
    axes[1].set_title("residual 1st LSV")
    plot_rsv_residual(result_dataset, axes[2], indices=[0])
    axes[2].set_xlabel("Wavelength (nm)")
    axes[2].set_title("residual 1st RSV")
    axes[2].get_legend().remove()
    axes[2].set_ylabel("")

    return fig, axes


fig, axes = plot_residual_and_svd(global_result.data["700TR1"])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)


# Simultaneous global analysis of TA of WL-PSI of Scy6803

### Inspect data

In [ ]:
from cycler import cycler
from glotaran.io import load_parameters, save_result
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme
from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.plot_traces import (
    plot_fitted_traces,
    select_plot_wavelengths,
)
from pyglotaran_extras.inspect import show_a_matrixes


## Global Analysis

### Used model and parameters

In [ ]:
global_model_path = "models/20230927model_PSI_TA_SCy6803WLglobal_with_DOAS.yml"


In [ ]:
global_parameters_path = "models/20230927model_PSI_TA_SCy6803WLglobal_with_DOAS.csv"
optimizedparameters = load_parameters(global_parameters_path)


#### Model file

In [ ]:
# Uncomment the following 2 lines to display the global model file in the notebook
# from glotaran.utils.ipython import display_file
# display_file(global_model_path, syntax="yaml")

# Alternatively (recommended), open the file in a text editor to see the model definition


#### Parameters file

In [ ]:
# Uncomment the next line and run the cell to print the starting values of the analysis
# These starting values have already been optimized, hence the name optimizedparameters

# optimizedparameters


### Create scheme and optimize it

In [ ]:
global_scheme = Scheme(
    model=global_model_path,  # type: ignore
    parameters=optimizedparameters,
    maximum_number_function_evaluations=5,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "670TR1": DATA_PATH1,
        "670TR2": DATA_PATH2,
        "700TR1": DATA_PATH3,
        "700TR2": DATA_PATH4,
    },  # type: ignore
)
global_scheme.validate()


In [ ]:
global_result = optimize(global_scheme, raise_exception=True)


To save the results of the optimization we can use the `save_result` command.

Because it saves *everything* it consumes about 50MB of disk space per save.

In [ ]:
save_result(
    result=global_result,
    result_path="results/20230821simul/result.yaml",
    allow_overwrite=True,
)


### Results and parameters

In [ ]:
# Just call the result to get the optimization result summary.
global_result
# For easier copy-and-paste try:
# print(global_result)


In [ ]:
# Access the result's `optimized_parameters` to print a markdown table of the optimized parameters:
global_result.optimized_parameters


### Q: what to do if the estimated rate constants are not in decreasing order?
then one should sort the estimated rate constants in decreasing order, and repeat the fit with those new starting values

### Amplitude matrices

In [ ]:
show_a_matrixes(global_result)


## Result plots

<sub>Note: The color scheme of the plots in this notebook may not match published figures.</sub>

## Fit quality

In [ ]:
global_result_TA = (
    global_result.data["670TR1"],
    global_result.data["670TR2"],
    global_result.data["700TR1"],
    global_result.data["700TR2"],
)
wavelengths = select_plot_wavelengths(global_result_TA, equidistant_wavelengths=True)
plot_fitted_traces(global_result_TA, wavelengths, linlog=True, linthresh=1);


The above command `plot_fitted_traces` is used to plot a selection of traces for a set of wavelengths (autogenerated using the `select_plot_wavelengths` function).
To show to make a manual selection of traces, and 'dress up the plot' see the code below, which reproduces Figure 2 of the paper.

In [ ]:
# Reproduction of Figure 2 of the paper
import warnings
from pyglotaran_extras.plotting.style import ColorCode as cc

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fig, ax_ = plot_fitted_traces(
        global_result_TA,
        [685, 700, 720, 760],
        linlog=True,
        linthresh=1,  # published figure uses 0.3 for easthetic reasons, but here 1 looks better
        axes_shape=(2, 2),
        figsize=(6, 4),
        title="",
        per_axis_legend=True,
        cycler=cycler(
            color=[
                cc.grey,
                cc.black,
                cc.grey,
                cc.black,
                cc.orange,
                cc.red,
                cc.orange,
                cc.red,
            ]
        ),
    )
    handles, labels = ax_.flatten()[0].get_legend_handles_labels()
    for i in range(len(handles)):
        if i == 1:
            labels[i] = "670 nm excitation"
        elif i == 5:
            labels[i] = "700 nm excitation"
        else:
            labels[i] = "_Hidden"
    for idx, ax in enumerate(ax_.flatten()):
        ax.set_ylabel(ax.title.get_text().replace("spectral = ", ""))
        if idx > 1:
            ax.set_xlabel("Time (ps)")
        else:
            ax.set_xlabel("")
        ax.set_title("")
        if ax.get_legend() is not None:
            ax.get_legend().remove()
        for line in ax.lines:
            line.set_linewidth(0.5)  # Set the line width here
    fig.legend(
        handles,
        labels,
        bbox_to_anchor=(0.5, -0.05),
        loc="lower center",
        ncol=len(handles),
    )
    fig.tight_layout()


## Overview 670 exc

In [ ]:
from pyglotaran_extras.plotting.style import ColorCode

plot_overview(
    global_result.data["670TR1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=2,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=[ "tab:grey","tab:orange",  ColorCode.cyan, ColorCode.green,"m", "y", "k","r", "b", "tab:purple"]
    ),use_svd_number=True,das_cycler=PlotStyle().cycler,svd_cycler=PlotStyle().cycler,
);


## Residual analysis of the 670 nm excitation TR1 data

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_residual import plot_residual
from pyglotaran_extras.plotting.plot_svd import plot_lsv_residual
from pyglotaran_extras.plotting.plot_svd import plot_rsv_residual


def plot_residual_and_svd(result_dataset):
    fig, axes = plt.subplots(1, 3, figsize=(10, 2))
    plot_residual(result_dataset, axes[0])
    axes[0].get_legend().remove()
    axes[0].set_ylabel("Wavelength (nm)")
    plot_lsv_residual(result_dataset, axes[1], indices=[0])
    axes[1].get_legend().remove()
    axes[1].set_ylabel("")
    axes[1].set_title("residual 1st LSV")
    plot_rsv_residual(result_dataset, axes[2], indices=[0])
    axes[2].set_xlabel("Wavelength (nm)")
    axes[2].set_title("residual 1st RSV")
    axes[2].get_legend().remove()
    axes[2].set_ylabel("")

    return fig, axes


fig, axes = plot_residual_and_svd(global_result.data["670TR1"])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)


In [ ]:
global_result.data["670TR2"]


## Residual analysis of all data

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_residual import plot_residual
from pyglotaran_extras.plotting.plot_svd import plot_lsv_residual
from pyglotaran_extras.plotting.plot_svd import plot_rsv_residual


def plot_svd_of_residual(result_dataset,result_dataset2,result_dataset3,result_dataset4,linlog,linthresh,index):
    fig, axes = plt.subplots(1, 2, figsize=(10, 2))
    custom_cycler = cycler(color=["tab:grey"])
    plot_lsv_residual(result_dataset, axes[0], indices=[index],linlog=linlog,linthresh=linthresh, cycler=custom_cycler)
    plot_lsv_residual(result_dataset2, axes[0], indices=[index],linlog=linlog,linthresh=linthresh)
    custom_cycler = cycler(color=["tab:orange"])
    plot_lsv_residual(result_dataset3, axes[0], indices=[index],linlog=linlog,linthresh=linthresh, cycler=custom_cycler)
    custom_cycler = cycler(color=["r"])
    plot_lsv_residual(result_dataset4, axes[0], indices=[index],linlog=linlog,linthresh=linthresh, cycler=custom_cycler)
    axes[0].set_xlabel("Time (ps)")
    axes[0].get_legend().remove()
    axes[0].set_ylabel("")
    axes[0].set_title("residual 1st LSV")
    custom_cycler = cycler(color=["tab:grey"])
    plot_rsv_residual(result_dataset, axes[1], indices=[index], cycler=custom_cycler)
    plot_rsv_residual(result_dataset2, axes[1], indices=[index])
    custom_cycler = cycler(color=["tab:orange"])
    plot_rsv_residual(result_dataset3, axes[1], indices=[index], cycler=custom_cycler)
    custom_cycler = cycler(color=["r"])
    plot_rsv_residual(result_dataset4, axes[1], indices=[index], cycler=custom_cycler)
    axes[1].set_xlabel("Wavelength (nm)")
    axes[1].set_title("residual 1st RSV")
    axes[1].get_legend().remove()
    axes[1].set_ylabel("")

    return fig, axes

fig, axes = plot_svd_of_residual(global_result.data["670TR1"],global_result.data["670TR2"],global_result.data["700TR1"],global_result.data["700TR2"],linlog=True,linthresh=1,index=0)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)


In [ ]:
global_result.data["700TR2"].residual_right_singular_vectors.isel(right_singular_value_index=0,spectral=[60,80])


In [ ]:
global_result.data["700TR2"].residual.plot(x="time")


In [ ]:
global_result.data["700TR2"].weighted_residual.plot(x="time")


In [ ]:
global_result.data["700TR1"].weighted_residual.plot(x="time")


In [ ]:
global_result.data["700TR2"].residual_right_singular_vectors.isel(spectral=slice(56,72),right_singular_value_index=0).plot()


### Q: why is weighted_residual_right_singular_vectors not visible in the print of the result object?

In [ ]:
global_result.data["700TR2"].weighted_residual_right_singular_vectors.isel(spectral=slice(56,72),right_singular_value_index=0).plot()


In [ ]:
global_result.data["700TR2"].weighted_residual_right_singular_vectors.isel(spectral=slice(56,72),right_singular_value_index=1).plot()


In [ ]:
global_result.data["700TR2"].residual_right_singular_vectors.isel(spectral=[60,80],right_singular_value_index=0)


In [ ]:
fig, axes = plot_svd_of_residual(global_result.data["670TR1"],global_result.data["670TR2"],global_result.data["700TR1"],global_result.data["700TR2"],linlog=True,linthresh=1,index=1)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[0].set_title("residual 2nd LSV")
axes[1].set_title("residual 2nd RSV")


In [ ]:
plot_overview(
    global_result.data["670TR2"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=2,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        # color=[ "tab:grey","tab:orange",  "c", "g","m", "y", "k","r", "b", "tab:purple"]
        color=[ "tab:grey","tab:orange",  ColorCode.cyan, ColorCode.green,"m", "y", "k","r", "b", "tab:purple"]
    ),use_svd_number=True,das_cycler=PlotStyle().cycler,svd_cycler=PlotStyle().cycler,
);


## Overview 700 exc

In [ ]:
plot_overview(
    global_result.data["700TR1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=2,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=["k","r", "g", "tab:purple"]
    ),use_svd_number=True,svd_cycler=PlotStyle().cycler,
);


In [ ]:
plot_overview(
    global_result.data["700TR2"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=2,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=["k","r", "g", "tab:purple"]
    ),use_svd_number=True,svd_cycler=PlotStyle().cycler,
);


## Residual analysis of the 700 nm excitation TR2 data

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_residual import plot_residual
from pyglotaran_extras.plotting.plot_svd import plot_lsv_residual
from pyglotaran_extras.plotting.plot_svd import plot_rsv_residual


def plot_residual_and_svd(result_dataset):
    fig, axes = plt.subplots(1, 3, figsize=(10, 2))
    plot_residual(result_dataset, axes[0])
    axes[0].get_legend().remove()
    axes[0].set_xlabel("Time (ps)")
    axes[0].set_ylabel("Wavelength (nm)")
    plot_lsv_residual(result_dataset, axes[1], indices=[0])
    axes[1].set_xlabel("Time (ps)")
    axes[1].get_legend().remove()
    axes[1].set_ylabel("")
    axes[1].set_title("residual 1st LSV")
    plot_rsv_residual(result_dataset, axes[2], indices=[0])
    axes[2].set_xlabel("Wavelength (nm)")
    axes[2].set_title("residual 1st RSV")
    axes[2].get_legend().remove()
    axes[2].set_ylabel("")

    return fig, axes


fig, axes = plot_residual_and_svd(global_result.data["700TR2"])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)


## Plot result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_das
from pyglotaran_extras.plotting.plot_spectra import plot_sas
from pyglotaran_extras.plotting.style import ColorCode

myFRLcolors = [ "tab:grey","tab:orange",  ColorCode.cyan, ColorCode.green,"m", "y", "k","r", "b", "tab:purple"]

custom_cycler = cycler(color=myFRLcolors)


def plot_concentration_and_spectra(result_dataset,result_dataset2):
    # fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    plot_concentrations(result_dataset, axes[0], center_λ=0, linlog=True, cycler=custom_cycler)
    plot_concentrations(result_dataset2, axes[0], center_λ=0, linlog=True, cycler=custom_cycler)
    plot_sas(result_dataset, axes[1], cycler=custom_cycler)
    plot_das(result_dataset, axes[2], cycler=custom_cycler)
    return fig, axes


fig, axes = plot_concentration_and_spectra(global_result.data["670TR1"],global_result.data["670TR2"])
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("EADS (mOD)")
axes[1].set_title("EADS")
axes[2].set_xlabel("Wavelength (nm)")
axes[2].set_ylabel("DADS (mOD)")
axes[2].set_title("DADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)


In [ ]:
# import matplotlib.pyplot as plt
# from cycler import cycler
# from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
# from pyglotaran_extras.plotting.plot_spectra import plot_sas
# from pyglotaran_extras.plotting.style import ColorCode

myFRLcolors = ["k","r", "g", "tab:purple"]

custom_cycler = cycler(color=myFRLcolors)


fig, axes = plot_concentration_and_spectra(global_result.data["700TR1"],global_result.data["700TR2"])
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("EADS (mOD)")
axes[1].set_title("EADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[2].set_xlabel("Wavelength (nm)")
axes[2].set_ylabel("DADS (mOD)")
axes[2].set_title("DADS")
axes[0].annotate("D", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("E", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[2].annotate("F", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)


## Coherent Artifact


In [ ]:
from pyglotaran_extras import plot_coherent_artifact

fig, axes = plot_coherent_artifact(global_result.data["670TR1"], time_range=(-0.3, 0.3), figsize=(10, 4))
axes[0].set_xlabel("Time (ps)")
axes[1].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("")
axes[0].annotate("A", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
fig.tight_layout()


In [ ]:
from pyglotaran_extras import plot_coherent_artifact

fig, axes = plot_coherent_artifact(global_result.data["700TR1"], time_range=(-0.3, 0.3), figsize=(10, 4))
axes[0].set_xlabel("Time (ps)")
axes[1].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("")
axes[0].annotate("A", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
fig.tight_layout()


## Overview of the estimated DOAS and phases of 670 nm excitation data

In [ ]:
from pyglotaran_extras import plot_doas
from pyglotaran_extras.plotting.style import ColorCode

fig, axes = plot_doas(
    global_result.data["670TR1"],
    damped_oscillation=["osc1"],
    time_range=(-0.3, 0.3),
    spectral=700,
    figsize=(15, 4),
    normalize=False
    # oscillation_type="sin",
    # center_λ=550,
)

# for vline_pos in [415, 460]:
#     axes[1].axvline(vline_pos, color="r", linewidth=1)
#     axes[2].axvline(vline_pos, color="r", linewidth=1)
# for vline_pos in [526]:
#     axes[1].axvline(vline_pos, color=ColorCode.green, linewidth=1)
#     axes[2].axvline(vline_pos, color=ColorCode.green, linewidth=1)
# for vline_pos in [393, 429, 479]:
#     axes[1].axvline(vline_pos, color="b", linewidth=1)
#     axes[2].axvline(vline_pos, color="b", linewidth=1)
axes[0].set_xlabel("Time (ps)")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[2].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[1].set_title("DOAS")
# axes[0].annotate("A", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
# axes[1].annotate("B", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
# axes[2].annotate("C", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[0].annotate("C", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[1].annotate("D", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[2].annotate("E", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)


In [ ]:
from glotaran.io import save_dataset
from glotaran.utils.io import create_clp_guide_dataset

for species in global_result.data["670TR2"].species:
    clp_guide = create_clp_guide_dataset(global_result.data["670TR2"], species.item())
    string_in_string = "guide/20231016global_670TR2_clp_{}.ascii".format(species.item())
    save_dataset(clp_guide.data, string_in_string,allow_overwrite=True)
